# **First Assignment of Algebra and Cryptography**



## **Exercise 2**

Bob intercepts from Alice the following encrypted message:

[427849968240759007228494978639775081809

498308250136673589542748543030806629941

925288105342943743271024837479707225255

95024328800414254907217356783906225740]

Alice knows that Bob uses RSA cryptosystem and his public key is (12398737, n) where

n = 956331992007843552652604425031376690367

Knowing that Alice and Bob agreed to use RSA cryptosystem to communicate in secret, each
message consist of a single letter which is encoded as: Space = 00, A = 11, B = 12, · · · , Z =
36, which message did Alice sent to Bob?

In [37]:
from IPython import get_ipython
from IPython.display import display
from sympy import factorint
import numpy as np


def is_prime(n):
  """
  Check if a number is prime.
  """
  if n <= 1:
      return False
  if n <= 3:
      return True
  if n % 2 == 0 or n % 3 == 0:
      return False
  i = 5
  while i * i <= n:
      if n % i == 0 or n % (i + 2) == 0:
          return False
      i = i + 6
  return True

def erastothene_sieve(n):
  R = int(n**(1/2))
  #A = Prime_less_root(int(R))

  for i in range(1, R+1):
    if is_prime(i) == False:
      return False
    if n % i == 0:
      return False
  return True

def Extended_Euclidean(a,b):
  old_r = a
  old_u = 1
  old_v = 0
  r = b
  u = 0
  v = 1

  while r != 0:
    quotient = old_r // r
    old_r, r = r, old_r - quotient * r
    old_u, u = u, old_u - quotient * u
    old_v, v = v, old_v - quotient * v

  d = old_r
  u = old_u
  v = v


  return u, v, d

def Euler_totient(n):
  """
  Calculates Euler's totient function (phi) for a given integer n.

  Args:
    n: An integer greater than 1.

  Returns:
    The value of Euler's totient function for n.
  """
  if n <= 1:
      return 0
  if erastothene_sieve(n):
    return n - 1  # If n is prime, phi(n) = n - 1

  factors = factorint(n)
  if len(factors) == 1:
    # If n is a power of a single prime (n = p^k)
    p, k = list(factors.items())[0]
    return p ** k - p ** (k - 1)
  else:
    # If n has multiple prime factors
    phi = 1
    for p, k in factors.items():
        phi *= (p ** k - p**(k-1))
    return phi


def inverse(a, n):
  u, v, d = Extended_Euclidean(a, n)
  if d != 1:
    return "a is not invertible in Z/nZ"
  return u % n


def decrypted(e, n, M):
  phi = Euler_totient(n)
  u, v, d = Extended_Euclidean(e, phi)
  if d!= 1:
    return
  secret_key = inverse(e, phi)
  decrypt = pow(M, secret_key, n)
  return decrypt

def numbers_to_letters(input_string):
    """Converts numbers (00-36) to letters (Space, A-Z)."""
    letter_map = {
        "00": " ",
        "11": "A", "12": "B", "13": "C", "14": "D", "15": "E", "16": "F", "17": "G",
        "18": "H", "19": "I", "20": "J", "21": "K", "22": "L", "23": "M", "24": "N",
        "25": "O", "26": "P", "27": "Q", "28": "R", "29": "S", "30": "T", "31": "U",
        "32": "V", "33": "W", "34": "X", "35": "Y", "36": "Z"
    }
    message = ""
    input_string = str(input_string)
    for i in range(0, len(input_string), 2):
          pair = input_string[i:i+2]
          #print(f"Processing pair: {pair}")

          for key, value in letter_map.items():
            if key == pair:
              message += value
              break
    return message




In [38]:
# Encrypted messages
Messages = [427849968240759007228494978639775081809,
            498308250136673589542748543030806629941,
            925288105342943743271024837479707225255,
            95024328800414254907217356783906225740,
            ]

# The modulus
n = 956331992007843552652604425031376690367
# Public key exponent
e = 12398737

Final_message = ""
# Decrypt and print the messages
for i in range(len(Messages)):
    decrypt = decrypted(e, n, Messages[i])
    #print(decrypt)
    print(numbers_to_letters(decrypt))
    Final_message += numbers_to_letters(decrypt)

print("=========================================================")
print("The message is: ", Final_message)
print("=========================================================")


THIS IS MY LETTER T
O THE WORLD THAT NE
VER WROTE TO ME EMI
LY DICKINSON
The message is:  THIS IS MY LETTER TO THE WORLD THAT NEVER WROTE TO ME EMILY DICKINSON


# Exercice 3

Create your own public key and private key for RSA cryptosystem. The two prime numbers
must have 600 digits and they have to be safe prime numbers. Then, Set up your own RSA
cryptosystem. Demonstrate how a message addressed to you can be encrypted and how you
can decrypt it using your private key.

In [2]:
#pip install gmpy2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.0 MB/s eta 0:00:00


In [39]:
from sympy import isprime, nextprime, factorint
import gmpy2
import random

### Function to generate a safe prime using strong primality test

In [65]:
import random
import sympy
from concurrent.futures import ThreadPoolExecutor, as_completed

def strong_pseudoprimality_test(N):
    """
    Strong pseudoprimality test (Miller-Rabin). Returns "composite" or "probably prime".
    """
    if N < 3 or N % 2 == 0:
        return "composite"

    e, m = 0, N - 1
    while m % 2 == 0:
        e += 1
        m //= 2

    # Randomly choose a base for the test
    x = random.randint(2, N - 2)
    if sympy.gcd(x, N) != 1:
        return "composite"

    y = pow(x, m, N)
    if y == 1:
        return "probably prime"

    for _ in range(e):
        if y == N - 1:
            return "probably prime"
        y = (y * y) % N

    return "composite"

def generate_safe_prime_t(n):
    """
    Generates a probable safe prime (p such that (p-1)/2 is also prime) with approximately n bits.
    """
    # Calculate limits using integer arithmetic only
    x = 2 ** n
    y0 = (x - 1) // (2 * n)  # Avoid floating-point calculations
    y1 = (2 ** (n - 1)) - 1

    with ThreadPoolExecutor() as executor:
        while True:
            l = random.randint(y0, y1)
            futures = {
                executor.submit(strong_pseudoprimality_test, l): "l",
                executor.submit(strong_pseudoprimality_test, 2 * l + 1): "2l + 1"
            }
            for future in as_completed(futures):
                result = future.result()
                if futures[future] == "l":
                    result_l = result
                else:
                    result_2l_plus_1 = result

            if result_l == "probably prime" and result_2l_plus_1 == "probably prime":
                return 2 * l + 1

            for j in range(n // 4, n // 2):
                l1_min = 2 ** j + 1
                l1_max = 2 ** (j + 1)

                if l1_min >= l1_max:
                    continue

                l1 = random.randint(l1_min, l1_max - 1)
                l2_min = (y0 + 1) // l1
                l2_max = y1 // l1

                if l2_min >= l2_max:
                    continue

                l2 = random.randint(l2_min, l2_max)

                # Use parallelism to check the primality of l1, l2, and 2*l1*l2 + 1
                futures = {
                    executor.submit(strong_pseudoprimality_test, l1): "l1",
                    executor.submit(strong_pseudoprimality_test, l2): "l2",
                    executor.submit(strong_pseudoprimality_test, 2 * l1 * l2 + 1): "2*l1*l2 + 1"
                }
                results = {}
                for future in as_completed(futures):
                    results[futures[future]] = future.result()

                if (results["l1"] == "probably prime" and
                    results["l2"] == "probably prime" and
                    results["2*l1*l2 + 1"] == "probably prime"):
                    return 2 * l1 * l2 + 1



# Test of the generated numbers

In [68]:
%%time
print("=========================================================")
p = generate_safe_prime_t(300)
print("p: ", p)
decimal_digits = len(str(p))
print(f"Number of decimal digits: {decimal_digits}")
print(factorint(p))
#print(factorint(p-1))

print("=========================================================")
q = generate_safe_prime_t(300)
print("q: ", q)
decimal_digits = len(str(q))
print(f"Number of decimal digits: {decimal_digits}")
print(factorint(q))
#print(factorint(q-1))
print("=========================================================")

p:  1217040076609411615839262227942218257476267789649527875069393536983025964696025516125817827
Number of decimal digits: 91
{1217040076609411615839262227942218257476267789649527875069393536983025964696025516125817827: 1}
q:  163088019152089316197019699901786434995486377038072109720482765947816686583923280041918767
Number of decimal digits: 90
{163088019152089316197019699901786434995486377038072109720482765947816686583923280041918767: 1}
CPU times: user 6min 17s, sys: 19.7 s, total: 6min 37s
Wall time: 6min 41s


# NB: This code taking long time, we choose to use in built function and parallelism

### Function to generate a safe prime using in built function

In [42]:
%%time

import gmpy2
import random
from concurrent.futures import ThreadPoolExecutor

def isprime(n):
    """Check if n is a prime number."""
    return gmpy2.is_prime(n)

def generate_safe_prime(n_bits=1992):
    """Generate a safe prime p where (p-1)/2 is prime."""
    x = gmpy2.mpz(2) ** n_bits  # x = 2^n
    ln_x = gmpy2.log(x)  # Natural logarithm of x
    y0 = (x - ln_x) / (2 * ln_x)  # Compute y0
    y1 = gmpy2.mpz(2) ** (n_bits - 1) - 1  # Compute y1

    while True:
        # Choose l uniformly at random in the range [y0, y1]
        l = random.randint(int(y0), int(y1))

        # Check if l and 2l + 1 are prime
        if isprime(l):
            p = 2 * l + 1

            if isprime(p) and len(str(p)) >= 600:  # Ensure at least 600 digits
                return p

# Function to generate two safe primes in parallel
def generate_two_safe_primes(n_bits=1992):
    with ThreadPoolExecutor() as executor:
        future1 = executor.submit(generate_safe_prime, n_bits)
        future2 = executor.submit(generate_safe_prime, n_bits)

        p = future1.result()
        q = future2.result()

    return p, q

# Generate two 600-digit safe primes
print("\nGenerating two 600-digit safe primes in parallel...")
p, q = generate_two_safe_primes()
print(f"\nGenerated safe prime 1: {p}")
print(f"\nGenerated safe prime 2: {q}")


Generating two 600-digit safe primes in parallel...

Generated safe prime 1: 343045481342836958005403432588425232453333816012904686943263837247177406593459493555666703082751840130731060715259573030506386525356591284595131563602031750832623330957231321812319300081157764151549879434561535786143620237471565383998260868132068834511882569614765161552148320605615460765671500337931232299738761509790549709413894343358376466526025094290140979802179035086092617776555030020740789370131807375512610818374793088232572636310418366123258555317074800016697002593110857799049146314698882985598863633799459446039461969110928630145157990326215378792511563150152002723387116808046300693719083

Generated safe prime 2: 18212116175933368248045143960411719959002499757523187700078488389901385985810359486572367630030944315550345169624671707524755605196839832392787491621921390373491399500469715710091003266828644946775766083500657682621459379111365830753189383791276691750844167422103096496248041987906018727833425668

# Test of the generated numbers

In [43]:
%%time
print("=========================================================")
#p = generate_safe_prime_t(92)
print("p: ", p)
decimal_digits = len(str(p))
print(f"Number of decimal digits: {decimal_digits}")
print(factorint(p))
print(factorint(p- 1))

print("=========================================================")
#q = generate_safe_prime_t(92)
print("q: ", q)
decimal_digits = len(str(q))
print(f"Number of decimal digits: {decimal_digits}")
print(factorint(q))
print(factorint(q-1))
print("=========================================================")

p:  343045481342836958005403432588425232453333816012904686943263837247177406593459493555666703082751840130731060715259573030506386525356591284595131563602031750832623330957231321812319300081157764151549879434561535786143620237471565383998260868132068834511882569614765161552148320605615460765671500337931232299738761509790549709413894343358376466526025094290140979802179035086092617776555030020740789370131807375512610818374793088232572636310418366123258555317074800016697002593110857799049146314698882985598863633799459446039461969110928630145157990326215378792511563150152002723387116808046300693719083
Number of decimal digits: 600
{3430454813428369580054034325884252324533338160129046869432638372471774065934594935556667030827518401307310607152595730305063865253565912845951315636020317508326233309572313218123193000811577641515498794345615357861436202374715653839982608681320688345118825696147651615521483206056154607656715003379312322997387615097905497094138943433583764665260250942901409798021

### We are testing the primality of  p and q generate

For this we choose 1992 bits

In [51]:
def letters_to_numbers(input_string):
    """Converts letters (Space, A-Z) to numbers (00-36)."""
    letter_map = { " ": "00", "A": "11", "B": "12", "C": "13", "D": "14", "E": "15", "F": "16", "G": "17",
        "H": "18", "I": "19", "J": "20", "K": "21", "L": "22", "M": "23", "N": "24",
        "O": "25", "P": "26", "Q": "27", "R": "28", "S": "29", "T": "30", "U": "31",
        "V": "32", "W": "33", "X": "34", "Y": "35", "Z": "36" }
    number_string = ""
    for char in input_string:
        if char in letter_map:
            number_string += letter_map[char]
        else:
            raise ValueError(f"Character '{char}' is not supported.")
    return int(number_string)


def numbers_to_letters(input_string):
    """Converts numbers (00-36) to letters (Space, A-Z)."""
    letter_map = { "00": " ", "11": "A", "12": "B", "13": "C", "14": "D", "15": "E", "16": "F", "17": "G",
        "18": "H", "19": "I", "20": "J", "21": "K", "22": "L", "23": "M", "24": "N",
        "25": "O", "26": "P", "27": "Q", "28": "R", "29": "S", "30": "T", "31": "U",
        "32": "V", "33": "W", "34": "X", "35": "Y", "36": "Z" }
    message = ""
    input_string = str(input_string)
    for i in range(0, len(input_string), 2):
        pair = input_string[i:i+2]
        for key, value in letter_map.items():
            if key == pair:
                message += value
                break
    return message

### Necessaries functions

In [52]:
def Extended_Euclidean(a,b):
  old_r = a
  old_u = 1
  old_v = 0
  r = b
  u = 0
  v = 1

  while r != 0:
    quotient = old_r // r
    old_r, r = r, old_r - quotient * r
    old_u, u = u, old_u - quotient * u
    old_v, v = v, old_v - quotient * v

  d = old_r
  u = old_u
  v = v


  return u, v, d


def inverse(a, n):
  u, v, d = Extended_Euclidean(a, n)
  if d != 1:
    return "a is not invertible in Z/nZ"
  return u % n

### Application example

In [53]:
# Compute n and Euler_totient(n)
n = p * q
phi_n = (p-1) * (q-1)


# Choose a public exponent e
e = 65537

# Compute the private exponent d
d = inverse(e, phi_n)

# Display the keys
print("=========================================================")
print("Public Key: (n, e) =", (n, e))
print("=========================================================")
print("Private Key: (n, d) =", (phi_n, d))
print("=========================================================")

# Example message to encrypt
message = "The following algorithm picks alternatingly random candidates for \
these two options until success"
print("=========================================================")
print("The input message is:", message)
print("=========================================================")
# Convert message to a number
m = letters_to_numbers(message.upper())

# Encrypt the message: c = m^e mod n
c = pow(m, e, n)

print("Encrypted message :", c)
print("=========================================================")

# Decrypt the message: m' = c^d mod n
m_decrypted = pow(c, d, n)
print(m_decrypted)

# Convert back to string
decrypted_message = numbers_to_letters(m_decrypted)
print("Decrypted message:", decrypted_message)


Public Key: (n, e) = (624758415984472944319877044251649471958292352773814267411618511519792153216512853215324019951700839675196459183088542825719384479390642899590109724301569297501726513395514161328619435336371908654299196834765569378175228326868306086075153747559457680507834580827419395967534902702680889041759515580977621036538951832953578600237935255184712068227126394434541372183273739639292492929237715915572293352740722198839926306333867447661640204921643730643938097308457207273020025007082217825896135930215507279109709537852707586628690524122014947973965724047013819094291002498252866677074167244249133348118156371095048692386606246846980298806733509134042364921351409603930362944548583203657190360427976642064109179200020687211120686819123695558975089233548442784320266929607559244402222375233392832178465464055639528160447615983575791857988570820518173100244850550557214398261746641330715896563377318231185085369292270083811599315993898824684961478076539254275978723998570691249316351937